In [ ]:
!pip install datasets -q
!pip install sentence_transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("pacovaldez/stackoverflow-questions",split="validation")
dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/pacovaldez___parquet/pacovaldez--stackoverflow-questions-77c9a43a92f08f84/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


Dataset({
    features: ['title', 'body', 'label'],
    num_rows: 785098
})

In [ ]:
texts = dataset['title']
print (len(texts))

785098


In [ ]:
texts[:10]

['How to dynamically instntiate a class?',
 'update value on button on:click event vue js',
 'Setinterval for div in jquery',
 'Phonegap twitter plugin for ios4',
 'Looping thru 2 set of data in puppet - optimalization',
 'Is this Dicelose correct for multiclass',
 'How to update UIProgressview in Tableviewcell?',
 'Add user in aws postgreSQL DB when user added to AD using lambda',
 'print char pointer value in c',
 'Antd Custom Radio Button Mega Option Form Component']

###Feature extraction

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [ ]:
sample = texts[5]
sample_enc = model.encode(sample)
print('Len of enc', len(sample_enc))
print(sample_enc)

Len of enc 384
[-0.34238416  0.37420255 -0.45176974 -0.9250324  -0.46697903 -0.08909241
  0.15029134 -0.23625584  0.23827915  0.091152    0.22503486 -0.09578213
 -0.32164347  0.49602672  0.4121493   0.09596958 -0.42052796  0.08749796
 -0.55316466  0.02788608  0.5204728  -0.21576327  0.3658779  -0.2348228
  0.02392516  0.241407   -0.5039559  -0.09328692  0.4293289   0.37273744
  0.3109426   0.9808594  -0.1090479  -0.11080169 -0.4497072   0.23855282
 -0.55143213  0.41394415 -0.03059197  0.5504758  -0.10439856  0.13134602
 -0.07704391  0.22947846 -0.2890695   0.20369743 -0.7639801   0.26500294
  0.16125569 -0.10006406  0.17438015 -0.28656796 -0.72924215  0.20886289
  0.2942986  -0.15818846 -0.12104984 -0.3150435  -0.13363765  0.09422016
 -0.04318074 -0.08944153 -0.01707537  0.39439744  0.3902581  -0.37725782
 -0.03140267  0.23960167 -0.060191    0.13326687 -0.21193138 -0.32110822
 -0.14727922 -0.02377565  0.30522373  0.08653908 -0.07893246 -0.13314138
  0.66175646 -0.01746193 -0.06526871 

In [ ]:
embeddings=model.encode(texts[:700])

###clustering

In [ ]:
from sklearn.cluster import KMeans
km=KMeans(n_clusters=5,random_state=42)
km.fit(embeddings)
cluster_labels=km.labels_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


###Search

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity as cs

def search(query, texts, embeddings):
    q_emb = model.encode([query])[0]
    sim_scores = cs([q_emb], embeddings)[0]
    results = [(texts[i], sim_scores[i]) for i in range(len(texts))]
    results.sort(key = lambda x: x[1], reverse = True)
    return results

In [ ]:
query = 'Module not found'
s_results = search(query, texts[:700], embeddings)

In [ ]:
from tqdm import tqdm

scores = []
titles = []

for i in range(len(tqdm(s_results))):
    scores.append(s_results[i][1])
    titles.append(s_results[i][0])

  0%|          | 0/700 [00:00<?, ?it/s]


In [ ]:
import pandas as pd
df=pd.DataFrame(
    {
        "Titles": titles,
        "Sim": scores
    }
)
df=df.sort_values('Sim',ascending=False)
df.head(10)

,Titles,Sim
0,ImportError: No module named 'parse',0.663350
1,Module build failed (from ./node_modules/babel...,0.549500
2,eslint + typescript + path in tsconfig: ESLint...,0.467615
3,How do I include a Perl module that's in a dif...,0.453819
4,Error whilst installing magento extension- pac...,0.438409
5,Entity Framework Cast not supported,0.410880
6,ASP.NET framework MVC5 empty project NuGet Pac...,0.408930
7,Failing to compile vlc for android,0.388735
8,Problems While Trying To Compile MacVim,0.387978
9,Eclipse Java Missing required source folder: '...,0.369784


###Deployment

In [ ]:
def deploy(error_text):
    s_results = search(error_text, texts[:700], embeddings)
    scores = []
    titles = []

    for i in range(len(tqdm(s_results))):
        scores.append(s_results[i][1])
        titles.append(s_results[i][0])

    df = pd.DataFrame({"Titles": titles,"Sim": scores})
    df = df.sort_values('Sim', ascending = False)
    print(df.head(10))

In [ ]:
x = input("Enter an error description.....")
deploy(str(x))

Enter an error description.....Import error


  0%|          | 0/700 [00:00<?, ?it/s]

                                              Titles       Sim
0               ImportError: No module named 'parse'  0.511569
1                       When to use absolute imports  0.488845
2  Angular: HttpClientModule import error (could ...  0.448119
3                 How to customize PHP import script  0.403576
4  Module build failed (from ./node_modules/babel...  0.395008
5          can someone tell me what this error mean?  0.389174
6  Error whilst installing magento extension- pac...  0.371819
7  SCSS compiling to CSS shows Error: Undefined m...  0.362897
8  eslint + typescript + path in tsconfig: ESLint...  0.362084
9  ASP.NET framework MVC5 empty project NuGet Pac...  0.343171
